In [1]:
import dask.dataframe as dd
# this should whatever your locomizer data file is
#%time temp = dd.read_csv("../data/sample/signals_set_2.csv")
import pandas as pd

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pyarrow.feather as ft

In [4]:
dtype={'Cited_Appln_id': 'float64',
       'Cited_pub_date': 'float64',
       'Citn_category': 'object',
       'Citn_lag_month': 'float64',
       'Citn_lag_year': 'float64'}

In [13]:
%time cite_epo = dd.read_csv('Citations/202001_EPO_CITATIONS.txt', delimiter='|', dtype = dtype)
%time cite_epo.head()

CPU times: user 117 ms, sys: 17 ms, total: 134 ms
Wall time: 213 ms
CPU times: user 5.4 s, sys: 467 ms, total: 5.87 s
Wall time: 7.93 s


,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_origin,Citn_category,Citn_lag_year,Citn_lag_month,PCT_Route
0,EP0000001,19781220,EP19780200013,16428854,DE2161506,19730614.0,DE,DE2161506,9912741.0,7,SEA,NaN,5.0,66.0,0
1,EP0000001,19781220,EP19780200013,16428854,FR2025459,19700911.0,FR,FR6942154,19548401.0,7,SEA,NaN,8.0,99.0,0
2,EP0000001,19781220,EP19780200013,16428854,US3532159,19701006.0,US,US3532159D,49957875.0,7,SEA,NaN,8.0,98.0,0
3,EP0000001,19781220,EP19780200013,16428854,US3568762,19710309.0,US,US3568762D,50023219.0,7,SEA,NaN,7.0,93.0,0
4,EP0000001,19781220,EP19780200013,16428854,US3913665,19751021.0,US,US40265573,50609500.0,7,SEA,NaN,3.0,38.0,0


In [5]:
%time cite_pct = dd.read_csv('Citations/202001_PCT_CITATIONS.txt', delimiter='|', dtype = dtype)
%time cite_pct.head()

CPU times: user 77.3 ms, sys: 5.25 ms, total: 82.5 ms
Wall time: 120 ms
CPU times: user 4.93 s, sys: 610 ms, total: 5.54 s
Wall time: 7.99 s


,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_origin,Citn_category,Citn_lag_year,Citn_lag_month,ISA
0,WO1978000001,19781019,WO1978000002,46895317,US3560400,19710202.0,US,US3560400D,50008254.0,3,ISR,A,7.0,92.0,US
1,WO1978000001,19781019,WO1978000002,46895317,US3597252,19710803.0,US,US3597252D,50072663.0,3,ISR,A,7.0,86.0,US
2,WO1978000001,19781019,WO1978000002,46895317,US3699050,19721017.0,US,US3699050D,50252625.0,3,ISR,A,6.0,72.0,US
3,WO1978000002,19781207,WO1978000035,22653893,DE1260751,19680208.0,DE,DEE0019771,11981374.0,8,ISR,NaN,10.0,130.0,EP
4,WO1978000002,19781207,WO1978000035,22653893,DE660199,19380519.0,DE,DEK0130767,12281964.0,8,ISR,NaN,40.0,487.0,EP


In [3]:
%time epo_pct = dd.read_csv('PATENTS/Patents/202001_EPO_PCT.txt', delimiter='|')
%time epo_pct.head()

CPU times: user 57.4 ms, sys: 7.3 ms, total: 64.7 ms
Wall time: 113 ms
CPU times: user 9.61 s, sys: 940 ms, total: 10.6 s
Wall time: 14.1 s


,app_nbr,pct_nbr
0,EP20000900148,WO2001050102
1,EP20000900214,WO2000041589
2,EP20000900314,WO2000040664
3,EP20000900384,WO2001052208
4,EP20000900567,WO2000042327


In [3]:
ipc = dd.read_csv('PATENTS/Patents/202001_EPO_IPC.txt', delimiter='|')
ipc.head()

,appln_id,prio_year,app_year,IPC
0,1,1999,2000,G06K007/00
1,1,1999,2000,G06K017/00
2,1,1999,2000,G06K019/077
3,1,1999,2000,H01R012/18
4,1,1999,2000,H04M001/02


In [3]:
dtype={'reg_share': 'float64'}
%time epo_inv = dd.read_csv('PATENTS/Patents/202001_EPO_Inv_reg.txt', delimiter='|', dtype=dtype)
%time epo_inv.head(10)

CPU times: user 87.8 ms, sys: 8.08 ms, total: 95.9 ms
Wall time: 125 ms
CPU times: user 3.66 s, sys: 493 ms, total: 4.15 s
Wall time: 6.03 s


,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share
0,EP20000103094,1,1037159,2,"Lipponen, Markku","Simo Kaarion katu 1 A 2,33720 Tampere",FI197,FI,1.0,0.250000
1,EP20000103094,1,1037159,3,"Laitinen, Timo","Peiponkatu 6,37830 Viiala",FI197,FI,1.0,0.250000
2,EP20000103094,1,1037159,4,"Aho, Ari","Elementinpolku 13 A 6,33720 Tampere",FI197,FI,1.0,0.250000
3,EP20000103094,1,1037159,5,"Knuutila, Jarno","Matti Tapion katu 1 F 17,33720 Tampere",FI197,FI,1.0,0.250000
4,EP20000107845,2,1024191,9,"Griffiths, Andrew David","28 Lilac Court, Cherry Hinton Road,Cambridge C...",UKH12,GB,1.0,0.166667
5,EP20000107845,2,1024191,10,"Hoogenboom, Hendricus Renerus Jacobus Mattheus","1 Hauxton Road, Little Shelford,Cambridge CB2 5JH",UKH12,GB,1.0,0.166667
6,EP20000107845,2,1024191,11,"Marks, James David","107 Ardmore,Kesington, CA 94707",US06001,US,1.0,0.166667
7,EP20000107845,2,1024191,12,"McCafferty, John","32 Wakelin Avenue,Sawston, Cambridgeshire CB2 4DA",UKH12,GB,1.0,0.166667
8,EP20000107845,2,1024191,13,"Winter, Gregory Paul","c/o Trinity College,Cambridge CB2 1TQ",UKH12,GB,1.0,0.166667
9,EP20000107845,2,1024191,14,"Grigg, Geoffrey Walter","352 Burns Bay Road, Lane Cove,Linley Point, NS...",AU1GS,AU,1.0,0.166667


In [2]:
dtype={'reg_share': 'float64', 'appln_id': 'float64'}
%time pct_inv = dd.read_csv('PATENTS/Patents/202001_PCT_Inv_reg.txt', delimiter='|', dtype=dtype)
%time pct_inv.head()

CPU times: user 92 ms, sys: 8.33 ms, total: 100 ms
Wall time: 147 ms
CPU times: user 4.26 s, sys: 518 ms, total: 4.78 s
Wall time: 6.61 s


,pct_nbr,internat_appln_nr,appln_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share
0,WO1978000001,WO1978US00002,46895317.0,"SAMANTA, Mrinmay",Washington D.C. 20013,US11001,US,1.0,1.0
1,WO1978000003,WO1978EP00001,15648845.0,"BRACH, Ulrich",D-5583 Zell/Mosel,DEB16,DE,1.0,1.0
2,WO1978000004,WO1978GB00001,22653861.0,"ARMITAGE, Arthur Advanced Chemical Equipment","Mirfield, West Yorkshire WF14 8BB",UKE44,GB,1.0,1.0
3,WO1978000005,WO1978SE00001,43420652.0,"MOSBACH, Klaus H.",S-244 02 Furulund,SE224,SE,1.0,1.0
4,WO1978000006,WO1978CH00002,6508282.0,"Erskine, William Graham",Morpeth Northumberland NE61 2YL,UKC21,GB,1.0,0.5


In [3]:
# Filter rows
filtered_pct_inv = pct_inv[pct_inv['ctry_code'] == 'GB']

# Write to a parquet file
filtered_pct_inv.to_parquet('PCT_INV_GB.parquet')


In [9]:
# Filter rows
filtered_epo_inv = epo_inv[epo_inv['ctry_code'] == 'GB']

# Write to a parquet file
filtered_epo_inv.to_parquet('EPO_INV_GB.parquet')


In [4]:
pct_inv_gb = dd.read_parquet('PCT_INV_GB.parquet')

In [5]:
epo_inv_gb = dd.read_parquet('EPO_INV_GB.parquet')

In [8]:
pct_inv_gb.head()

,pct_nbr,internat_appln_nr,appln_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share
2,WO1978000004,WO1978GB00001,22653861.0,"ARMITAGE, Arthur Advanced Chemical Equipment","Mirfield, West Yorkshire WF14 8BB",UKE44,GB,1.0,1.0
4,WO1978000006,WO1978CH00002,6508282.0,"Erskine, William Graham",Morpeth Northumberland NE61 2YL,UKC21,GB,1.0,0.5
25,WO1979000002,WO1978GB00002,22653862.0,"MARTIN, Archer, John, Porter","Elstree, Hertfordshire",UKH23,GB,1.0,0.5
26,WO1979000002,WO1978GB00002,22653862.0,"HAMPSON, Frank","Lewes, Sussex",UKJ22,GB,1.0,0.5
83,WO1979000070,WO1978GB00007,22653867.0,"Joseph, Simon Philip",London S.W.13,UKI11,GB,1.0,1.0


In [11]:
epo_inv_gb.head()

,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share
4,EP20000107845,2,1024191,9,"Griffiths, Andrew David","28 Lilac Court, Cherry Hinton Road,Cambridge C...",UKH12,GB,1.0,0.166667
5,EP20000107845,2,1024191,10,"Hoogenboom, Hendricus Renerus Jacobus Mattheus","1 Hauxton Road, Little Shelford,Cambridge CB2 5JH",UKH12,GB,1.0,0.166667
7,EP20000107845,2,1024191,12,"McCafferty, John","32 Wakelin Avenue,Sawston, Cambridgeshire CB2 4DA",UKH12,GB,1.0,0.166667
8,EP20000107845,2,1024191,13,"Winter, Gregory Paul","c/o Trinity College,Cambridge CB2 1TQ",UKH12,GB,1.0,0.166667
206,EP20050251525,85,1577898,327,"Chang, Simon Cambridge Silicon Radio Limited","Unit 400 Science Park Milton Road,Cambridge CB...",UKH12,GB,1.0,1.000000


In [15]:
app_nbr_series_epo = epo_inv_gb['app_nbr'].compute()

# Filter 'cite_epo' DataFrame
filtered_cite_epo = cite_epo[cite_epo['Citing_app_nbr'].isin(app_nbr_series_epo)]
filtered_cite_epo.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_origin,Citn_category,Citn_lag_year,Citn_lag_month,PCT_Route
10,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,SEA,NaN,54.0,659.0,0
365,EP0000096,19781220,EP19780300026,16429236,FR1484507,19670609.0,FR,FR66859,18661071.0,5,SEA,NaN,11.0,138.0,0
366,EP0000096,19781220,EP19780300026,16429236,FR2062186,19710625.0,FR,FR7033543,19593924.0,5,SEA,NaN,7.0,90.0,0
367,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,SEA,NaN,5.0,67.0,0
368,EP0000096,19781220,EP19780300026,16429236,US3308098,19670307.0,US,US5278760,46463362.0,5,SEA,NaN,11.0,141.0,0


In [7]:
app_nbr_series_pct = pct_inv_gb['pct_nbr'].compute()

# Filter 'cite_epo' DataFrame
filtered_cite_pct = cite_pct[cite_pct['Citing_app_nbr'].isin(app_nbr_series_pct)]
filtered_cite_pct.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_origin,Citn_category,Citn_lag_year,Citn_lag_month,ISA
145,WO1979000004,19790111,WO1978000004,22653864,DE2023727,19701119.0,DE,DE2023727,9802470.0,8,ISR,NaN,9.0,98.0,EP
146,WO1979000004,19790111,WO1978000004,22653864,DE2311656,19730920.0,DE,DE2311656,10004540.0,8,ISR,NaN,6.0,64.0,EP
147,WO1979000004,19790111,WO1978000004,22653864,FR1381839,19641214.0,FR,FR962620,18957938.0,8,ISR,NaN,15.0,169.0,EP
148,WO1979000004,19790111,WO1978000004,22653864,FR2093021,19720128.0,FR,FR7018992,19578237.0,8,ISR,NaN,7.0,84.0,EP
149,WO1979000004,19790111,WO1978000004,22653864,FR1465710,19670113.0,FR,FR40926,20859865.0,8,ISR,A,12.0,144.0,EP


In [16]:
# Write the filtered DataFrame to a Parquet file
filtered_cite_epo.to_parquet('filtered_cite_epo.parquet')

In [22]:
# Write the filtered DataFrame to a Parquet file
filtered_cite_pct.to_parquet('filtered_cite_pct.parquet')

In [17]:
filtered_cite_epo_gb = filtered_cite_epo[filtered_cite_epo['Cited_App_auth'] == 'GB']

# Write to a parquet file
filtered_cite_epo_gb.to_parquet('filtered_cite_epo_GB.parquet')

In [8]:
filtered_cite_pct_gb = filtered_cite_pct[filtered_cite_pct['Cited_App_auth'] == 'GB']

# Write to a parquet file
filtered_cite_pct_gb.to_parquet('filtered_cite_pct_GB.parquet')

In [7]:
#Filter epo_pct to have only the rows in pct_inv_gb
app_nbr_series_pct = pct_inv_gb['pct_nbr'].compute()
fltered_epo_pct = epo_pct[epo_pct['pct_nbr'].isin(app_nbr_series_pct)]

In [9]:
fltered_epo_pct.to_parquet('filtered_epo_pct.parquet')

In [10]:
filtered_epo_pct = dd.read_parquet('filtered_epo_pct.parquet')

In [11]:
# Step 1: Merge pct_inv_gb with filtered_epo_pct to get corresponding app_nbr
pct_inv_gb = pct_inv_gb.merge(filtered_epo_pct, on='pct_nbr', how='left')
pct_inv_gb = pct_inv_gb.drop(['pct_nbr', 'internat_appln_nr'], axis=1)
#pct_inv_gb.head()
#len(pct_inv_gb)

In [12]:
# Step 2: Filter pct_inv_gb to only keep rows where app_nbr is not in epo_inv_gb
app_nbr_epo = epo_inv_gb['app_nbr'].compute()
pct_inv_gb = pct_inv_gb[~pct_inv_gb['app_nbr'].isin(app_nbr_epo)]
pct_inv_gb.head()

,appln_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share,app_nbr
144,22670316.0,"PANTRIDGE, James Francis","Dunmurry, Co. Antrim",UKN03,GB,1.0,0.5,EP19790900569
145,22670316.0,"ANDERSON, John","Holywood, Co. Down",UKN03,GB,1.0,0.5,EP19790900569
220,22670436.0,"BUNYAN, Thomas Walter",London SE10,UKI11,GB,1.0,1.0,EP19790901193
552,22728089.0,"FRASER, Thomas Munro","Great Bowden, Leicestershire",UKF22,GB,1.0,0.5,EP19810900568
553,22728089.0,"WEBB, Anthony",Leicester,GBZZZ,GB,1.0,0.5,EP19810900568


In [13]:
# Step 3: Append pct_inv_gb to epo_inv_gb
epo_pct_inv = dd.concat([epo_inv_gb, pct_inv_gb], axis=0)
epo_pct_inv.head()

,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share
4,EP20000107845,2.0,1024191.0,9.0,"Griffiths, Andrew David","28 Lilac Court, Cherry Hinton Road,Cambridge C...",UKH12,GB,1.0,0.166667
5,EP20000107845,2.0,1024191.0,10.0,"Hoogenboom, Hendricus Renerus Jacobus Mattheus","1 Hauxton Road, Little Shelford,Cambridge CB2 5JH",UKH12,GB,1.0,0.166667
7,EP20000107845,2.0,1024191.0,12.0,"McCafferty, John","32 Wakelin Avenue,Sawston, Cambridgeshire CB2 4DA",UKH12,GB,1.0,0.166667
8,EP20000107845,2.0,1024191.0,13.0,"Winter, Gregory Paul","c/o Trinity College,Cambridge CB2 1TQ",UKH12,GB,1.0,0.166667
206,EP20050251525,85.0,1577898.0,327.0,"Chang, Simon Cambridge Silicon Radio Limited","Unit 400 Science Park Milton Road,Cambridge CB...",UKH12,GB,1.0,1.000000


In [14]:
epo_pct_inv.to_parquet('EPO_PCT_INV.parquet')

In [16]:
len(epo_inv_gb)

385085

In [15]:
len(epo_pct_inv)

502314

In [4]:
filtered_cite_epo= dd.read_parquet('filtered_cite_epo.parquet')
filtered_cite_pct= dd.read_parquet('filtered_cite_pct.parquet')

In [5]:
#Filter epo_pct to have only the rows in filtered_cite_epo
app_nbr_series_pct = filtered_cite_pct['Citing_app_nbr'].compute()
fltered_epo_pct_cite = epo_pct[epo_pct['pct_nbr'].isin(app_nbr_series_pct)]

In [10]:
fltered_epo_pct_cite.to_parquet('filtered_epo_pct_cite.parquet')

In [6]:
filtered_epo_pct_cite = dd.read_parquet('filtered_epo_pct_cite.parquet')

In [12]:
len(filtered_epo_pct_cite)

75320

In [7]:
# Step 1: Merge filtered_cite_pct with filtered_epo_pct_cite to get corresponding app_nbr
filtered_cite_pct = filtered_cite_pct.merge(filtered_epo_pct_cite, left_on='Citing_app_nbr', right_on='pct_nbr', how='left')
filtered_cite_pct.head()
# filtered_cite_pct = filtered_cite_pct.drop(['pct_nbr', 'internat_appln_nr'], axis=1)
#pct_inv_gb.head()
#len(pct_inv_gb)

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_origin,Citn_category,Citn_lag_year,Citn_lag_month,ISA,app_nbr,pct_nbr
0,WO1979000004,19790111,WO1978000004,22653864,DE2023727,19701119.0,DE,DE2023727,9802470.0,8,ISR,None,9.0,98.0,EP,EP19780900003,WO1978000004
1,WO1979000004,19790111,WO1978000004,22653864,DE2311656,19730920.0,DE,DE2311656,10004540.0,8,ISR,None,6.0,64.0,EP,EP19780900003,WO1978000004
2,WO1979000004,19790111,WO1978000004,22653864,FR1381839,19641214.0,FR,FR962620,18957938.0,8,ISR,None,15.0,169.0,EP,EP19780900003,WO1978000004
3,WO1979000004,19790111,WO1978000004,22653864,FR2093021,19720128.0,FR,FR7018992,19578237.0,8,ISR,None,7.0,84.0,EP,EP19780900003,WO1978000004
4,WO1979000004,19790111,WO1978000004,22653864,FR1465710,19670113.0,FR,FR40926,20859865.0,8,ISR,A,12.0,144.0,EP,EP19780900003,WO1978000004


In [16]:
filtered_cite_epo.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_origin,Citn_category,Citn_lag_year,Citn_lag_month,PCT_Route
10,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,SEA,None,54.0,659.0,0
365,EP0000096,19781220,EP19780300026,16429236,FR1484507,19670609.0,FR,FR66859,18661071.0,5,SEA,None,11.0,138.0,0
366,EP0000096,19781220,EP19780300026,16429236,FR2062186,19710625.0,FR,FR7033543,19593924.0,5,SEA,None,7.0,90.0,0
367,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,SEA,None,5.0,67.0,0
368,EP0000096,19781220,EP19780300026,16429236,US3308098,19670307.0,US,US5278760,46463362.0,5,SEA,None,11.0,141.0,0


In [9]:
filtered_cite_pct = filtered_cite_pct[['Citing_pub_nbr', 'Citing_pub_date', 'app_nbr', 'Citing_appln_id','Cited_pub_nbr','Cited_pub_date','Cited_App_auth','Cited_App_nbr','Cited_Appln_id','Cit_Total','Citn_lag_year','Citn_lag_month']]
filtered_cite_pct.head()

,Citing_pub_nbr,Citing_pub_date,app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
0,WO1979000004,19790111,EP19780900003,22653864,DE2023727,19701119.0,DE,DE2023727,9802470.0,8,9.0,98.0
1,WO1979000004,19790111,EP19780900003,22653864,DE2311656,19730920.0,DE,DE2311656,10004540.0,8,6.0,64.0
2,WO1979000004,19790111,EP19780900003,22653864,FR1381839,19641214.0,FR,FR962620,18957938.0,8,15.0,169.0
3,WO1979000004,19790111,EP19780900003,22653864,FR2093021,19720128.0,FR,FR7018992,19578237.0,8,7.0,84.0
4,WO1979000004,19790111,EP19780900003,22653864,FR1465710,19670113.0,FR,FR40926,20859865.0,8,12.0,144.0


In [12]:
filtered_cite_pct = filtered_cite_pct.rename(columns = {'app_nbr':'Citing_app_nbr'})
filtered_cite_epo = filtered_cite_epo[['Citing_pub_nbr', 'Citing_pub_date', 'Citing_app_nbr', 'Citing_appln_id','Cited_pub_nbr','Cited_pub_date','Cited_App_auth','Cited_App_nbr','Cited_Appln_id','Cit_Total','Citn_lag_year','Citn_lag_month']]
filtered_cite_epo.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
10,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,54.0,659.0
365,EP0000096,19781220,EP19780300026,16429236,FR1484507,19670609.0,FR,FR66859,18661071.0,5,11.0,138.0
366,EP0000096,19781220,EP19780300026,16429236,FR2062186,19710625.0,FR,FR7033543,19593924.0,5,7.0,90.0
367,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,5.0,67.0
368,EP0000096,19781220,EP19780300026,16429236,US3308098,19670307.0,US,US5278760,46463362.0,5,11.0,141.0


In [23]:
filtered_cite_pct.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
0,WO1979000004,19790111,EP19780900003,22653864,DE2023727,19701119.0,DE,DE2023727,9802470.0,8,9.0,98.0
1,WO1979000004,19790111,EP19780900003,22653864,DE2311656,19730920.0,DE,DE2311656,10004540.0,8,6.0,64.0
2,WO1979000004,19790111,EP19780900003,22653864,FR1381839,19641214.0,FR,FR962620,18957938.0,8,15.0,169.0
3,WO1979000004,19790111,EP19780900003,22653864,FR2093021,19720128.0,FR,FR7018992,19578237.0,8,7.0,84.0
4,WO1979000004,19790111,EP19780900003,22653864,FR1465710,19670113.0,FR,FR40926,20859865.0,8,12.0,144.0


In [13]:
# Step 2: Filter FILTERED_cite_pct to only keep rows where Citing_app_nbr is not in filtered_cite_epo
app_cite_epo = filtered_cite_epo['Citing_app_nbr'].compute()
filtered_cite_pct = filtered_cite_pct[~filtered_cite_pct['Citing_app_nbr'].isin(app_cite_epo)]
filtered_cite_pct.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
623,WO1980001422,19800710,EP19790900569,43451753,None,NaN,None,None,NaN,0,NaN,NaN
638,WO1980001434,19800710,EP19790900569,46930941,DE2754335,19780608.0,DE,DE2754335,10321991.0,6,2.0,25.0
639,WO1980001434,19800710,EP19790900569,46930941,DE2757841,19790607.0,DE,DE2757841,10325648.0,6,1.0,13.0
640,WO1980001434,19800710,EP19790900569,46930941,DE2800426,19781123.0,DE,DE2800426,10332179.0,6,2.0,20.0
641,WO1980001434,19800710,EP19790900569,46930941,US4011052,19770308.0,US,US62090975,52279747.0,6,3.0,40.0


In [14]:
# Step 3: Append filtered_cite_pct to filtered_cite_epo
filtered_cite_epo_pct = dd.concat([filtered_cite_epo, filtered_cite_pct], axis=0)
filtered_cite_epo_pct.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
10,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,54.0,659.0
365,EP0000096,19781220,EP19780300026,16429236,FR1484507,19670609.0,FR,FR66859,18661071.0,5,11.0,138.0
366,EP0000096,19781220,EP19780300026,16429236,FR2062186,19710625.0,FR,FR7033543,19593924.0,5,7.0,90.0
367,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,5.0,67.0
368,EP0000096,19781220,EP19780300026,16429236,US3308098,19670307.0,US,US5278760,46463362.0,5,11.0,141.0


In [15]:
len(filtered_cite_epo)

632269

In [16]:
len(filtered_cite_epo_pct)

941707

In [17]:
filtered_cite_epo_pct.to_parquet('EPO_PCT_CITED.parquet')

In [18]:
filtered_cite_epo_gb= dd.read_parquet('filtered_cite_epo_GB.parquet')
filtered_cite_pct_gb= dd.read_parquet('filtered_cite_pct_GB.parquet')

In [19]:
#Filter epo_pct to have only the rows in filtered_cite_epo
app_nbr_series_pct_gb = filtered_cite_pct_gb['Citing_app_nbr'].compute()
fltered_epo_pct_cite_gb = epo_pct[epo_pct['pct_nbr'].isin(app_nbr_series_pct_gb)]

In [20]:
fltered_epo_pct_cite_gb.to_parquet('filtered_epo_pct_cite_GB.parquet')

In [21]:
filtered_epo_pct_cite_gb = dd.read_parquet('filtered_epo_pct_cite_GB.parquet')

In [22]:
len(filtered_epo_pct_cite_gb)

11455

In [23]:
# Step 1: Merge filtered_cite_pct with filtered_epo_pct_cite to get corresponding app_nbr
filtered_cite_pct_gb = filtered_cite_pct_gb.merge(filtered_epo_pct_cite_gb, left_on='Citing_app_nbr', right_on='pct_nbr', how='left')
filtered_cite_pct_gb.head()
# filtered_cite_pct = filtered_cite_pct.drop(['pct_nbr', 'internat_appln_nr'], axis=1)
#pct_inv_gb.head()
#len(pct_inv_gb)

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_origin,Citn_category,Citn_lag_year,Citn_lag_month,ISA,app_nbr,pct_nbr
0,WO1979000004,19790111,WO1978000004,22653864,GB901203,19620718.0,GB,GB2056658,21936419.0,8,ISR,None,17.0,198.0,EP,EP19780900003,WO1978000004
1,WO1979000004,19790111,WO1978000004,22653864,GB1353919,19740522.0,GB,GB1353919D,23359768.0,8,ISR,None,5.0,56.0,EP,EP19780900003,WO1978000004
2,WO1979000032,19790125,WO1978000004,6508283,GB821016,19590930.0,GB,GB351458,21064988.0,11,ISR,None,20.0,232.0,EP,EP19780900003,WO1978000004
3,WO1979000063,19790222,WO1978000006,22653866,GB975295,19641111.0,GB,GB4204062,22491820.0,8,ISR,A,15.0,171.0,EP,EP19780900005,WO1978000006
4,WO1979000216,19790503,WO1978000006,29105620,GB1001028,19650811.0,GB,GB3171462,22286345.0,9,ISR,None,14.0,165.0,JP,EP19780900005,WO1978000006


In [24]:
filtered_cite_pct_gb = filtered_cite_pct_gb[['Citing_pub_nbr', 'Citing_pub_date', 'app_nbr', 'Citing_appln_id','Cited_pub_nbr','Cited_pub_date','Cited_App_auth','Cited_App_nbr','Cited_Appln_id','Cit_Total','Citn_lag_year','Citn_lag_month']]
filtered_cite_pct_gb.head()

,Citing_pub_nbr,Citing_pub_date,app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
0,WO1979000004,19790111,EP19780900003,22653864,GB901203,19620718.0,GB,GB2056658,21936419.0,8,17.0,198.0
1,WO1979000004,19790111,EP19780900003,22653864,GB1353919,19740522.0,GB,GB1353919D,23359768.0,8,5.0,56.0
2,WO1979000032,19790125,EP19780900003,6508283,GB821016,19590930.0,GB,GB351458,21064988.0,11,20.0,232.0
3,WO1979000063,19790222,EP19780900005,22653866,GB975295,19641111.0,GB,GB4204062,22491820.0,8,15.0,171.0
4,WO1979000216,19790503,EP19780900005,29105620,GB1001028,19650811.0,GB,GB3171462,22286345.0,9,14.0,165.0


In [25]:
filtered_cite_pct_gb = filtered_cite_pct_gb.rename(columns = {'app_nbr':'Citing_app_nbr'})
filtered_cite_epo_gb = filtered_cite_epo_gb[['Citing_pub_nbr', 'Citing_pub_date', 'Citing_app_nbr', 'Citing_appln_id','Cited_pub_nbr','Cited_pub_date','Cited_App_auth','Cited_App_nbr','Cited_Appln_id','Cit_Total','Citn_lag_year','Citn_lag_month']]
filtered_cite_epo_gb.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
10,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,54.0,659.0
367,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,5.0,67.0
866,EP0000229,19790110,EP19780200056,16428894,GB1413491,19751112.0,GB,GB2053973,21935542.0,2,4.0,38.0
925,EP0000240,19790110,EP19780300009,16429221,GB837680,19600615.0,GB,GB3400056,22349055.0,6,19.0,223.0
930,EP0000241,19790110,EP19780300013,16429224,GB1151690,19690514.0,GB,GB3813467,22437702.0,6,10.0,116.0


In [26]:
# Step 2: Filter FILTERED_cite_pct to only keep rows where Citing_app_nbr is not in filtered_cite_epo
app_cite_epo_gb = filtered_cite_epo_gb['Citing_app_nbr'].compute()
filtered_cite_pct_gb = filtered_cite_pct_gb[~filtered_cite_pct_gb['Citing_app_nbr'].isin(app_cite_epo_gb)]
filtered_cite_pct_gb.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
0,WO1979000004,19790111,EP19780900003,22653864,GB901203,19620718.0,GB,GB2056658,21936419.0,8,17.0,198.0
1,WO1979000004,19790111,EP19780900003,22653864,GB1353919,19740522.0,GB,GB1353919D,23359768.0,8,5.0,56.0
2,WO1979000032,19790125,EP19780900003,6508283,GB821016,19590930.0,GB,GB351458,21064988.0,11,20.0,232.0
3,WO1979000063,19790222,EP19780900005,22653866,GB975295,19641111.0,GB,GB4204062,22491820.0,8,15.0,171.0
4,WO1979000216,19790503,EP19780900005,29105620,GB1001028,19650811.0,GB,GB3171462,22286345.0,9,14.0,165.0


In [27]:
# Step 3: Append filtered_cite_pct to filtered_cite_epo
filtered_cite_epo_pct_gb = dd.concat([filtered_cite_epo_gb, filtered_cite_pct_gb], axis=0)
filtered_cite_epo_pct_gb.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
10,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,54.0,659.0
367,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,5.0,67.0
866,EP0000229,19790110,EP19780200056,16428894,GB1413491,19751112.0,GB,GB2053973,21935542.0,2,4.0,38.0
925,EP0000240,19790110,EP19780300009,16429221,GB837680,19600615.0,GB,GB3400056,22349055.0,6,19.0,223.0
930,EP0000241,19790110,EP19780300013,16429224,GB1151690,19690514.0,GB,GB3813467,22437702.0,6,10.0,116.0


In [28]:
len(filtered_cite_epo_gb)

48950

In [29]:
len(filtered_cite_epo_pct_gb)

70432

In [30]:
filtered_cite_epo_pct_gb.to_parquet('EPO_PCT_CITED_GB.parquet')

In [8]:
# pct_series = epo_pct['pct_nbr'].compute()
# filtered_ipc = pct_ipc[pct_ipc['pct_nbr'].isin(pct_series)]

In [6]:
# filtered_ipc.head()

In [5]:
print(len(pct_ipc))
print(len(epo_pct))

12601935
3549526


In [ ]:
# epo_pct_ipc = epo_pct.merge(pct_ipc, on='pct_nbr', how='left')
# epo_pct_ipc.head()

In [4]:
# Step 1: Keep only the first three characters of each IPC code
ipc['IPC'] = ipc['IPC'].str.slice(0, 3)

# Step 2: Remove duplicate IPC codes for the same patent
ipc_class = ipc.drop_duplicates(subset=['appln_id', 'IPC'])  


In [5]:
ipc_class = ipc_class.drop(['prio_year'], axis=1)

ipc_class.head()

,appln_id,app_year,IPC
0,1,2000,G06
3,1,2000,H01
4,1,2000,H04
8,2,1992,C07
18,2,1992,C12


In [ ]:
# len(ipc_new)

In [5]:
# epo_pct_inv = dd.read_parquet('data/EPO_PCT_INV.parquet')

In [6]:
# #Filter epo_pct to have only the rows in epo_pct_inv
# app_series = epo_pct_inv['app_nbr'].compute()
# filtered_epo_pct_onepo = epo_pct[epo_pct['app_nbr'].isin(app_series)]

In [7]:
# filtered_epo_pct_onepo.to_parquet('data_process/EPO_PCT_on_EPO.parquet')

In [5]:
# filtered_epo_pct_onepo = dd.read_parquet('data_process/EPO_PCT_on_EPO.parquet')

In [8]:
# epo_pct_ipc = filtered_epo_pct_onepo.merge(ipc_new, on='pct_nbr', how='left')
# epo_pct_ipc.head()

,app_nbr,pct_nbr,app_year,IPC
0,EP20000900684,WO2000041812,2000,B02
1,EP20000900750,WO2000043724,2000,F41
2,EP20000901250,WO2000045173,2000,G01
3,EP20000901250,WO2000045173,2000,C07
4,EP20000901671,WO2000046469,2000,E04


In [9]:
# epo_pct_ipc.to_parquet('data_process/EPO_PCT_IPC_Detail.parquet')

In [41]:
ipc_class.to_parquet('data_process/EPO_IPC_Class.parquet')

In [39]:
ipc_class = dd.read_parquet('data_process/EPO_IPC_Class.parquet')

In [40]:
ipc_class_pd = ipc_class.compute()

In [52]:
ipc_class_pd.head()

,appln_id,app_year,IPC
0,1,2000,G06
3,1,2000,H01
4,1,2000,H04
8,2,1992,C07
18,2,1992,C12


In [3]:
# epo_pct_ipc = dd.read_parquet('data_process/EPO_PCT_IPC_Detail.parquet')

In [ ]:
# # Step 1: Keep only the first three characters of each IPC code
# ipc_section = ipc_class_pd.copy()
# ipc_section['IPC'] = ipc_section['IPC'].str.slice(0, 1)

# # Step 2: Remove duplicate IPC codes for the same patent
# ipc_section = ipc_section.drop_duplicates(subset=['appln_id', 'IPC'])  

In [5]:
# ipc_section.to_parquet('data_process/EPO_PCT_IPC_Section.parquet')

In [35]:
# ipc_section.head()

,appln_id,app_year,IPC
0,1,2000,G
3,1,2000,H
8,2,1992,C
24,2,1992,G
28,3,2000,G


In [8]:
epo_pct_inv = dd.read_parquet('data/EPO_PCT_INV.parquet')

In [9]:
len(epo_pct_inv)

502314

In [6]:
# # Group by 'app_nbr' and keep the first 'person_id' for each group
# epo_pct_inv = epo_pct_inv.groupby('app_nbr').apply(lambda x: x.nlargest(1, 'person_id'), meta={'x': 'f8', 'y': 'f8'}).reset_index(drop=True).compute()
# epo_pct_inv.head(10)

In [10]:
# Convert the Dask dataframe to a Pandas dataframe
epo_pct_inv_pd = epo_pct_inv.compute()

# Keep the first 'person_id' for each 'app_nbr' in the Pandas dataframe
epo_pct_inv_pd = epo_pct_inv_pd.groupby('app_nbr').first().reset_index()

In [11]:
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_12621/3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [19]:
# epo_pct_inv_pd.head(10)

In [12]:
len(epo_pct_inv_pd)

256737

In [15]:
import re

# regex pattern for UK postcodes
postcode_regex = r"[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2}"

# keep only rows where 'address' matches the postcode pattern
inv_pandas = epo_pct_inv_pd[epo_pct_inv_pd['address'].str.contains(postcode_regex, na=False)]


In [17]:
len(inv_pandas)

223392

In [31]:
inv_pandas.head()

,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share
app_nbr,,,,,,,,,
EP19780100400,16427452.0,522.0,47138519.0,"Turner, George Frederick Albert","'Monks Acres' Woolmongers Lane Blackmore,Ingatestone Essex CM4 0JX",UKH33,GB,1.0,1.0
EP19780200076,16428913.0,237.0,2217720.0,"Ellson, Allan Henry","c/o Philips Patent Department Berkshire House,168-173 High Holborn London, WC1V 7AQ",UKI11,GB,1.0,1.0
EP19780200236,16429060.0,1659.0,2217967.0,"Brook, Neil Rodney","C/O PHILIPS Ltd. PATENT DEPT. Berkshire house,168-173 High Holborn London, WC1V 7AQ",UKI11,GB,1.0,0.5
EP19780300002,16429215.0,3.0,2218230.0,"Hatch, George","23 Grimshaw Avenue,Failsworth, Manchaster M35 9JT",UKD32,GB,1.0,1.0
EP19780300016,16429227.0,5702.0,2218261.0,"Lucas, James Frederick","9, New Road,South Darenth Kent. DA4 9AT",UKJ42,GB,1.0,1.0


In [13]:
epo_pct_ipc_inv_class = epo_pct_inv_pd.merge(ipc_class_pd, on='appln_id', how='left')
epo_pct_ipc_inv_class.head(10)

,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share,app_year,IPC
0,EP19780100247,16427314.0,344.0,2214446.0,"Bruton, Edwin John","9 Dunheved Road,Launceston, Cornwall",UKK30,GB,1.0,1.0,1978.0,E06
1,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,A61
2,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C07
3,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C12
4,EP19780100400,16427452.0,522.0,47138519.0,"Turner, George Frederick Albert","'Monks Acres' Woolmongers Lane Blackmore,Ingatestone Essex CM4 0JX",UKH33,GB,1.0,1.0,1978.0,G03
5,EP19780100463,16427512.0,559.0,2214885.0,"Wilkinson, Samuel","12 Bevington Road,Beckenham, Kent",UKJ42,GB,1.0,1.0,1978.0,A61
6,EP19780100463,16427512.0,559.0,2214885.0,"Wilkinson, Samuel","12 Bevington Road,Beckenham, Kent",UKJ42,GB,1.0,1.0,1978.0,C07
7,EP19780100492,16427540.0,576.0,2214948.0,"Barber, Bernard","3 Little Street,Yardley Hastings Northants",UKJ22,GB,1.0,1.0,1978.0,B29
8,EP19780100492,16427540.0,576.0,2214948.0,"Barber, Bernard","3 Little Street,Yardley Hastings Northants",UKJ22,GB,1.0,1.0,1978.0,F16
9,EP19780100645,16427679.0,788.0,2215277.0,"Jones, Dennis","Rosemount Alpraham,Tarporley Cheshire",UKD63,GB,1.0,1.0,1978.0,E01


In [14]:
# epo_pct_ipc_inv_sec = epo_pct_inv_pd.merge(ipc_section, on='appln_id', how='left')
# epo_pct_ipc_inv_sec.head()

In [ ]:
# epo_pct_ipc_inv_class.to_parquet('data/INV_IPC_Class.parquet')

In [15]:
len(epo_pct_ipc_inv_class)

366330

In [16]:
mis = epo_pct_ipc_inv_class['IPC'].isnull().sum()
mis

62093

In [17]:
epo_pct_ipc_inv_class = epo_pct_ipc_inv_class.dropna(subset=['IPC'])

In [18]:
len(epo_pct_ipc_inv_class)

304237

In [19]:
epo_pct_ipc_inv_class.to_csv('INV_IPC_CLASS.csv', index=False)


In [20]:
df = pd.read_csv('INV_IPC_CLASS.csv')
df.head()

,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share,app_year,IPC
0,EP19780100247,16427314.0,344.0,2214446.0,"Bruton, Edwin John","9 Dunheved Road,Launceston, Cornwall",UKK30,GB,1.0,1.0,1978.0,E06
1,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,A61
2,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C07
3,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C12
4,EP19780100400,16427452.0,522.0,47138519.0,"Turner, George Frederick Albert","'Monks Acres' Woolmongers Lane Blackmore,Ingatestone Essex CM4 0JX",UKH33,GB,1.0,1.0,1978.0,G03


In [21]:
df['IPC_Section'] = df['IPC'].str[0]
df.head()

,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share,app_year,IPC,IPC_Section
0,EP19780100247,16427314.0,344.0,2214446.0,"Bruton, Edwin John","9 Dunheved Road,Launceston, Cornwall",UKK30,GB,1.0,1.0,1978.0,E06,E
1,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,A61,A
2,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C07,C
3,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C12,C
4,EP19780100400,16427452.0,522.0,47138519.0,"Turner, George Frederick Albert","'Monks Acres' Woolmongers Lane Blackmore,Ingatestone Essex CM4 0JX",UKH33,GB,1.0,1.0,1978.0,G03,G


In [32]:
df = df.drop('IPC', axis=1)

# Remove duplicate rows
df = df.drop_duplicates()
df.head()

,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share,app_year,IPC_Section
0,EP19780100247,16427314.0,344.0,2214446.0,"Bruton, Edwin John","9 Dunheved Road,Launceston, Cornwall",UKK30,GB,1.0,1.0,1978.0,E
1,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,A
2,EP19780100395,16427449.0,520.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1.0,1.0,1978.0,C
4,EP19780100400,16427452.0,522.0,47138519.0,"Turner, George Frederick Albert","'Monks Acres' Woolmongers Lane Blackmore,Ingatestone Essex CM4 0JX",UKH33,GB,1.0,1.0,1978.0,G
5,EP19780100463,16427512.0,559.0,2214885.0,"Wilkinson, Samuel","12 Bevington Road,Beckenham, Kent",UKJ42,GB,1.0,1.0,1978.0,A


In [33]:
df.to_csv('INV_IPC_SECTION.csv', index=False)

In [34]:
ipc_sec = df[['appln_id','IPC_Section']]
ipc_sec.head()

,appln_id,IPC_Section
0,16427314.0,E
1,16427449.0,A
2,16427449.0,C
4,16427452.0,G
5,16427512.0,A


In [23]:
cite = dd.read_parquet('data/EPO_PCT_CITED_GB.parquet')

In [24]:
cite_pd = cite.compute()
cite_pd.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month
10,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,54.0,659.0
367,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,5.0,67.0
866,EP0000229,19790110,EP19780200056,16428894,GB1413491,19751112.0,GB,GB2053973,21935542.0,2,4.0,38.0
925,EP0000240,19790110,EP19780300009,16429221,GB837680,19600615.0,GB,GB3400056,22349055.0,6,19.0,223.0
930,EP0000241,19790110,EP19780300013,16429224,GB1151690,19690514.0,GB,GB3813467,22437702.0,6,10.0,116.0


In [30]:
len(cite_pd)

70432

In [26]:
mis = cite_pd['Cited_Appln_id'].isnull().sum()
mis

0

In [41]:
cite_pd_1 = cite_pd.merge(ipc_class_pd, left_on='Citing_appln_id', right_on='appln_id', how='left')
cite_pd_1.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month,appln_id,app_year,IPC
0,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,54.0,659.0,16429215.0,1978.0,E05
1,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,5.0,67.0,16429236.0,1978.0,C08
2,EP0000229,19790110,EP19780200056,16428894,GB1413491,19751112.0,GB,GB2053973,21935542.0,2,4.0,38.0,16428894.0,1978.0,A01
3,EP0000229,19790110,EP19780200056,16428894,GB1413491,19751112.0,GB,GB2053973,21935542.0,2,4.0,38.0,16428894.0,1978.0,C07
4,EP0000240,19790110,EP19780300009,16429221,GB837680,19600615.0,GB,GB3400056,22349055.0,6,19.0,223.0,16429221.0,1978.0,C07


In [42]:
len(cite_pd_1)

97519

In [45]:
mis = cite_pd_1['IPC'].isnull().sum()
mis

0

In [44]:
cite_pd_1 =cite_pd_1.dropna(subset=['IPC'])

In [46]:
len(cite_pd_1)

76032

In [48]:
cite_pd_1 = cite_pd_1.rename(columns={'app_year': 'Citing_app_year', 'IPC': 'Citing_IPC_Class'})
cite_pd_1 = cite_pd_1.drop(['appln_id'], axis = 1)
cite_pd_1.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month,Citing_app_year,Citing_IPC_Class
0,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,54.0,659.0,1978.0,E05
1,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,5.0,67.0,1978.0,C08
2,EP0000229,19790110,EP19780200056,16428894,GB1413491,19751112.0,GB,GB2053973,21935542.0,2,4.0,38.0,1978.0,A01
3,EP0000229,19790110,EP19780200056,16428894,GB1413491,19751112.0,GB,GB2053973,21935542.0,2,4.0,38.0,1978.0,C07
4,EP0000240,19790110,EP19780300009,16429221,GB837680,19600615.0,GB,GB3400056,22349055.0,6,19.0,223.0,1978.0,C07


In [51]:
cite_pd_2 = cite_pd.merge(ipc_class_pd, left_on='Cited_Appln_id', right_on='appln_id', how='left')
cite_pd_2.head()

,Citing_pub_nbr,Citing_pub_date,Citing_app_nbr,Citing_appln_id,Cited_pub_nbr,Cited_pub_date,Cited_App_auth,Cited_App_nbr,Cited_Appln_id,Cit_Total,Citn_lag_year,Citn_lag_month,appln_id,app_year,IPC
0,EP0000003,19781220,EP19780300002,16429215,GB209638,19240117.0,GB,GB724923,21194796.0,1,54.0,659.0,NaN,NaN,NaN
1,EP0000096,19781220,EP19780300026,16429236,GB1317774,19730523.0,GB,GB1317774D,23348110.0,5,5.0,67.0,NaN,NaN,NaN
2,EP0000229,19790110,EP19780200056,16428894,GB1413491,19751112.0,GB,GB2053973,21935542.0,2,4.0,38.0,NaN,NaN,NaN
3,EP0000240,19790110,EP19780300009,16429221,GB837680,19600615.0,GB,GB3400056,22349055.0,6,19.0,223.0,NaN,NaN,NaN
4,EP0000241,19790110,EP19780300013,16429224,GB1151690,19690514.0,GB,GB3813467,22437702.0,6,10.0,116.0,NaN,NaN,NaN


In [50]:
mis = cite_pd_2['IPC'].isnull().sum()
mis

76032

In [55]:
value_exists = ipc_class_pd['appln_id'].isin(['23348110']).any()


In [56]:
value_exists

False

In [57]:
cite_pd_1.to_csv('Citing_IPC_Class.csv', index=False)